In [60]:
from fileinput import filename
import pandas as pd
import os
import glob
import re

# Função para extrair número e complemento
# Função para extrair número e complemento corretamente
def extract_number_and_complement(endereco):
    if not isinstance(endereco, str):
        return None, None, None
    
    # Captura número da casa (1 a 5 dígitos isolados)
    numero = re.search(r'\b(\d{1,5})\b', endereco)
    numero = numero.group(1) if numero else None
    
    # Captura complementos válidos
    complemento = re.search(r'(?i)\b(AND\s*\d+|Apt[o]?\s*\d+|Ap\s*\d+|Sala?\s*\d+|Sal\s*\d+|SL\s*\d+)\b', endereco)
    complemento = complemento.group(0).strip() if complemento else None
    
    # Remove número e complemento do endereço
    endereco_limpo = re.sub(r'\b\d{1,5}\b', '', endereco)  # Remove número
    if complemento:
        endereco_limpo = re.sub(re.escape(complemento), '', endereco_limpo, flags=re.IGNORECASE)
    
    # Limpa espaços e pontuações extras
    endereco_limpo = re.sub(r'[^\w\s]', '', endereco_limpo).strip()
    
    return numero, complemento, endereco_limpo

path = 'C:\\Users\\edinocencio\\DataClenupCmsCrm\\DataSources\\'
file_name = 'PMD-CMS.xlsx'
full_file_name = path + file_name
sheet_name = 'after synch'

df_master = pd.read_excel(full_file_name, sheet_name=sheet_name)

HEADERS = df_master.iloc[0, :].tolist()

df_master.columns = HEADERS
df_master = df_master.iloc[1:,0:13]

In [64]:
df_master[['Numero', 'Complemento', 'LesseeStreet']] = df_master['LesseeStreet'].apply(
    lambda x: pd.Series(extract_number_and_complement(x))
)

In [71]:
COLUMNS = [
    'CompanyNumber', 'Branch', 
    'ContractNumber', 'Bearbeitungsstand', 
    'Rueckstand', 'LesseeStreet','Numero', 
    'Complemento','LesseePostCode', 'LesseeCity',
    'LesseCountry', 'LesseeCounty',
    'LesseeState', 'LesseeName']

df_master = df_master.loc[:,COLUMNS]

In [73]:
DE_PARA_CEPS = pd.read_excel(full_file_name, sheet_name='CEPS')

In [79]:
DE_PARA_CEPS.head()

,CEP_OLD,NUM_CARACT_OLD,CEP_NEW,NUM_CARACT_NEW
0,01331-010,9,01331-010,9
1,74250-010,9,74250-010,9
2,86990-000,9,86990-000,9
3,66093-180,9,66093-180,9
4,04530-001,9,04530-001,9


In [76]:
df_master = df_master.merge(DE_PARA_CEPS,left_on='LesseePostCode',right_on='CEP_OLD',how='left')


In [ ]:
COLUMNS = [
    'CompanyNumber', 'Branch', 
    'ContractNumber', 'Bearbeitungsstand', 
    'Rueckstand', 'CEP_NEW','Numero', 
    'Complemento','LesseePostCode', 'LesseeCity',
    'LesseCountry', 'LesseeCounty',
    'LesseeState', 'LesseeName']

df_master = df_master.loc[ : , COLUMNS ]

In [ ]:
path = 'C:\\Users\\edinocencio\\DataClenupCmsCrm\\DataSources\\finalizados\\'
file_name = 'base_cep_consolidada.xlsx'

full_file_name = path + file_name
BASE_CEP_API = pd.read_excel(full_file_name)

In [86]:
COLUMNS = [
    'cep_validado', 'logradouro',
    'bairro', 'cidade', 'uf', 'Estado'
]

BASE_CEP_API = BASE_CEP_API.loc[ : , COLUMNS ]

In [88]:
df_master = df_master.merge(BASE_CEP_API, how='left', left_on='CEP_NEW', right_on='cep_validado')

In [91]:
COLUMNS = [
    'CompanyNumber', 'LesseeName','Branch',
    'ContractNumber', 'Bearbeitungsstand',
    'Rueckstand', 'CEP_NEW', 'logradouro',
    'Numero', 'Complemento', 'bairro', 'cidade',
    'uf', 'Estado'
]
df_master = df_master.loc[ : , COLUMNS ]

In [ ]:
df_master

In [93]:
path = 'C:\\Users\\edinocencio\\DataClenupCmsCrm\\DataSources\\finalizados\\'
file_name = 'enderecos_processados.xlsx'
full_file_name = path + file_name

df_master.to_excel(full_file_name, index=False)